In [1]:
import requests
import GEOparse
import xml.etree.ElementTree as ET
import pandas 
from pandas import DataFrame as DF
from dcicutils import ff_utils
from functions.notebook_functions import *

In [2]:
#Getting excel file
readExcel = pandas.read_excel(r'/Users/luisa/Documents/4DN/Wrangling/Hi-C Datasets.xlsx')
df = DF(readExcel, columns = ['order', 'title','link','submitter','score','grouping','famous','details'])


In [3]:
All_GEO_list = df['link']
scores_list = df['score']
no_pubmed_list = []
parseGEO_error_list = []
organism_error_list = []
pubmed_GEO_list = []
pubmed_Score_list = []
no_pubmed_Score_list = []
geo_title_col = []
geo_authors_col = []

In [4]:
#New Table
pubID_col = []
Title_col = []
Authors_col = []
Human_col = []
Mouse_col = []
Keyword_col = []
year_col = []
status_col = []
link_col = []

In [5]:
x = -1
for GEOacc in All_GEO_list:
    x = x + 1
    print(GEOacc)
    #Get the GEO information from GEO accession
    print("Get the GEO information from GEO accession")
    try:
        gse = GEOparse.get_GEO(GEOacc)
    except:
        parseGEO_list.append(GEOacc)
        pubID_col.append('N/A')
        Title_col.append('N/A')
        Authors_col.append('N/A')
        Keyword_col.append('N/A')
        year_col.append('N/A')
        Mouse_col.append('N/A')
        Human_col.append('N/A')
        continue
    #Get metadata
    metadata = gse.metadata
    #metadata.keys()
    #Get Data of interest
    try:
        pubID_i = gse.metadata['pubmed_id']
    except:
        try:
            geo_title = gse.metadata['title']
        except:
            geo_title = 'N/A'
        try:
            geo_authors = gse.metadata['contributor']
        except:
            geo_authors = 'N/A'
            
        no_pubmed_Score_list.append(scores_list[x])
        no_pubmed_list.append(GEOacc)
        geo_title_col.append(geo_title)
        geo_authors_col.append(geo_authors)
        
        continue
    pubID = str(pubID_i[0])
    pubID_col.append(pubID)
    
    #Getting infor from pubmed using the pubmed ID
    
    
    #Getting xml format
    pubmed_URL_xml = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubID +"&retmode=xml"
    #print(pubmed_URL_xml)
    response = requests.get(pubmed_URL_xml)
    tree = ET.fromstring(response.content)
#     with open('file2.xml', 'wb') as f:
#         f.write(response.content)
#     tree = ET.parse('file2.xml')
    
    #The Data is stored in the child of the root
    # child = tree.getchildren()
    # print(child)
#     root = tree.getroot()
    for i in tree:
        child = i
    
    #The data is stored in the child of the root
    #The info of interest is in the medlineCitation level
    medlineCitation = 'MedlineCitation'
    
    #This are the elements of interest in this datalevel:
    Article = 'Article'
    KeywordList = 'KeywordList' #each element only has a single item. no index needed with for loop
    
    #Elements of Interest in Article level:
    AuthorList = 'AuthorList'  # there are several, each item has last name index = 0, first name = 1, and abbrev = 2
    ArticleTitle = 'ArticleTitle' # there is only one object so index = 0
    ArticleDate = 'ArticleDate' # there are several, each item has a Year index = 0, Month index = 0, and day index = 3
    
    Author_List = []
    keyword_list = []
    
    for i in child:
        if medlineCitation in str(i):
            for item in i:
                if Article in str(item):
                    for ele in item:
                        if ArticleTitle in str(ele):
                            Article_Title = ele.text
                            Title_col.append(Article_Title)
                        
                        if ArticleDate in str(ele):
                            Article_Year = ele[0].text
                            year_col.append(Article_Year)
                            
                        if AuthorList in str(ele):
                            for author in ele:
                                if len(author) >= 2:
                                    Lname = author[0].text
                                    Iname = author[1].text
                                    complName = Lname + ', ' + Iname
                                else:
                                    Lname = author[0].text
                                    complName = Lname
                                Author_List.append(complName)
                        
                if KeywordList in str(item):
                    for ele in item:
                        keyword = ele.text
                        keyword_list.append(keyword)
    
    Authors_col.append(Author_List)
    Keyword_col.append(keyword_list)
    pubmed_GEO_list.append(GEOacc)
    pubmed_Score_list.append(scores_list[x])
    
    #Getting Info from the GEO website
    print("Getting Info from the GEO website")
    URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + GEOacc
    print(URL_GEO_web) 
    
    response2 = requests.get(URL_GEO_web)
    my_file = response2.content.decode()

    
    #Organism Key Word:
    mouse1 = 'Mus musculus'
    human1 = 'Homo sapiens'

    if mouse1 in my_file:
        is_mouse = 'yes'
    else:
        is_mouse = 'no'
    Mouse_col.append(is_mouse)
            
    if human1 in my_file:
        is_human = 'yes'
    else:
        is_human = 'no'
    Human_col.append(is_human)
    

03-Dec-2018 12:19:57 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:19:57 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:19:57 INFO GEOparse - Parsing ./GSE98119_family.soft.gz: 
03-Dec-2018 12:19:57 DEBUG GEOparse - DATABASE: GeoMiame
03-Dec-2018 12:19:57 DEBUG GEOparse - SERIES: GSE98119
03-Dec-2018 12:19:57 DEBUG GEOparse - PLATFORM: GPL13112
03-Dec-2018 12:19:57 DEBUG GEOparse - PLATFORM: GPL17021
03-Dec-2018 12:19:58 DEBUG GEOparse - PLATFORM: GPL18573
03-Dec-2018 12:19:58 DEBUG GEOparse - PLATFORM: GPL19057
03-Dec-2018 12:19:58 DEBUG GEOparse - PLATFORM: GPL21273
03-Dec-2018 12:19:58 DEBUG GEOparse - PLATFORM: GPL21493
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587372
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587373
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587375
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587376
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587378
03-Dec-2018 12:19:58 DEBUG GEOparse 

GSE98119
Get the GEO information from GEO accession


03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587391
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587396
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587397
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587398
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587399
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587400
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587401
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587402
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587403
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587404
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587405
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587406
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587407
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587408
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587409
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587411
03-Dec-2018 12:19:58 DEBUG GEOparse - SAMPLE: GSM2587412
03-Dec-2018 12:19:58 DEBUG GEOp

03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139011
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139012
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139013
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139014
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139015
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139016
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139017
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139018
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139019
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139020
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139021
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139022
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139023
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139024
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139025
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139026
03-Dec-2018 12:19:59 DEBUG GEOparse - SAMPLE: GSM3139027
03-Dec-2018 12:19:59 DEBUG GEOp

03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139155
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139156
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139157
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139158
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139159
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139160
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139161
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139162
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139163
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139164
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139165
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139166
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139167
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139168
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139169
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139170
03-Dec-2018 12:20:00 DEBUG GEOparse - SAMPLE: GSM3139171
03-Dec-2018 12:20:00 DEBUG GEOp

...

Getting Info from the GEO website
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE20027


03-Dec-2018 12:28:52 DEBUG utils - Directory ./ already exists. Skipping.
03-Dec-2018 12:28:52 INFO GEOparse - File already exist: using local version.
03-Dec-2018 12:28:52 INFO GEOparse - Parsing ./GSE18350_family.soft.gz: 
03-Dec-2018 12:28:52 DEBUG GEOparse - DATABASE: GeoMiame
03-Dec-2018 12:28:52 DEBUG GEOparse - SERIES: GSE18350
03-Dec-2018 12:28:52 DEBUG GEOparse - PLATFORM: GPL570


GSE18350
Get the GEO information from GEO accession


03-Dec-2018 12:28:54 DEBUG GEOparse - SAMPLE: GSM458070
03-Dec-2018 12:28:54 DEBUG GEOparse - SAMPLE: GSM458071


Getting Info from the GEO website
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE18350


In [ ]:
#Make sure the lists the same number of elements

In [52]:
print(len(pubmed_GEO_list))
print(len(pubID_col))
print(len(Authors_col))
#Technique_col = []
print(len(Human_col))
print(len(Mouse_col))
print(len(Keyword_col))
print(len(year_col))
print(len(no_pubmed_list))
print(len(parseGEO_error_list))
print(len(pubmed_Score_list))
print(len(no_pubmed_Score_list))
print(len(geo_title_col))
print(len(geo_authors_col))

226
226
226
226
226
226
162
27
0
226
27
27
27


In [ ]:
#Check if publications in the list are in the portal

In [8]:
# get key from file
my_key = get_key('default')
# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'fourfront-hotseat')

In [9]:
q = '/search/?type=Publication'
results = ff_utils.search_metadata(q, key=my_key)

In [10]:
res_title_list=[]
res_uuid_list =[]
for i in results:
    res_title_list.append(i['title'])
    res_uuid_list.append(i['uuid'])

In [11]:
print(len(res_title_list))
print(len(res_uuid_list))

36
36


In [ ]:
print(res_title_list)

In [12]:
#Get the index of the publications that are already in the portal
index_list = []
for title in Title_col:
    if title in res_title_list:
        x = -1
        for ele in res_title_list:
            x = x + 1
            if title == ele:
                status = 'submitted'
                url = 'https://data.4dnucleome.org/publications/' + res_uuid_list[x]
                break
                   
    else:
        status = 'no_submitted'
        url = ''
    status_col.append(status)      
    link_col.append(url)
    
        

In [ ]:
#Export to excel

In [13]:
#This is the table of GEO accessions that have a publication
COLUMN = ['GEO-Accession','PMID','Title','Authors','Human','Mouse','Keywords', 'Score', 'Status','Link to Portal']
Dic = {'GEO-Accession':pubmed_GEO_list,'PMID':pubID_col,'Title':Title_col,'Authors':Authors_col,'Human':Human_col,'Mouse':Mouse_col,'Keywords':Keyword_col,'Score':pubmed_Score_list,'Status':status_col,'Link to Portal':link_col}

In [14]:
new_df = pandas.DataFrame(Dic, columns=COLUMN)


In [16]:
#This is the table of GEO accessions that do not have a publication
COLUMN_t2 = ['GEO-Accession','Title','Contributors']
Dic_t2 = {'GEO-Accession':no_pubmed_list,'Title':geo_title_col,'Contributors':geo_authors_col}

In [22]:
new_df_t2 = pandas.DataFrame(Dic_t2, columns=COLUMN_t2)

In [25]:
writer = pandas.ExcelWriter('results4.xlsx')

In [26]:
new_df.to_excel(writer,'sheet1')

In [27]:
new_df_t2.to_excel(writer,'sheet2')

In [28]:
writer.save()

In [ ]:
#Filter repeated PMID

In [40]:
#Getting excel file
readExcel = pandas.read_excel(r'/Users/luisa/Documents/Github/dcicwrangling/notebooks/Luisa_scripts/results5.xlsx')
new_table_df = DF(readExcel, columns = COLUMN)

In [41]:
#The PMID should be sorted
new_geo_column = []
new_PMID_column = []
new_Title_column = []
new_Authors_column = []
new_human_col = []
new_mouse_col = []
new_keywords_Col = []
new_Score_col = []
new_status_col = []
new_link_col = []
x = 0
ct = 0
new_list = []
while x <= 225:
    new_list.append(new_table_df['GEO-Accession'][ct])
    new_PMID_column.append(new_table_df['PMID'][ct])
    new_Title_column.append(new_table_df['Title'][ct])
    new_Authors_column.append(new_table_df['Authors'][ct])
    new_human_col.append(new_table_df['Human'][ct])
    new_mouse_col.append(new_table_df['Mouse'][ct])
    new_keywords_Col.append(new_table_df['Keywords'][ct])
    new_Score_col.append(new_table_df['Score'][ct])
    new_status_col.append(new_table_df['Status'][ct])
    new_link_col.append(new_table_df['Link to Portal'][ct])
    ct = ct + 1
    if ct != 226:
        while new_table_df['PMID'][x] == new_table_df['PMID'][ct]:
            new_list.append(new_table_df['GEO-Accession'][ct])
            ct = ct + 1
            if ct == 226:
                break
    x = ct
    new_geo_column.append(new_list)
    new_list = []


In [42]:
print(len(new_geo_column))

136


In [39]:
print(new_geo_column)

[['GSE107939', 'GSE107940'], ['GSE118514'], ['GSE109671'], ['GSE103477', 'GSE113703'], ['GSE96033'], ['GSE111720', 'GSE115933', 'GSE112046', 'GSE112045', 'GSE112043', 'GSE112041', 'GSE111726', 'GSE111724', 'GSE111723', 'GSE111722', 'GSE111721', 'GSE115932'], ['GSE105028'], ['GSE115980'], ['GSE100720'], ['GSE114242'], ['GSE99991'], ['GSE105776', 'GSE105918'], ['GSE99051', 'GSE97928', 'GSE98012'], ['GSE101921'], ['GSE109965'], ['GSE114121'], ['GSE102998', 'GSE102997'], ['GSE114747', 'GSE114746', 'GSE114745', 'GSE114744'], ['GSE98119', 'GSE82144'], ['GSE98443'], ['GSE89663'], ['GSE100832'], ['GSE110619'], ['GSE92291', 'GSE92290'], ['GSE104427', 'GSE98632'], ['GSE90034', 'GSE90033'], ['GSE100037'], ['GSE80795', 'GSE80799'], ['GSE103053'], ['GSE107726'], ['GSE79422'], ['GSE104129'], ['GSE100835'], ['GSE93995', 'GSE93996', 'GSE93994', 'GSE93993'], ['GSE108102'], ['GSE102740'], ['GSE96553'], ['GSE74055'], ['GSE99946', 'GSE99943'], ['GSE101715'], ['GSE102884'], ['GSE100569'], ['GSE76505'], ['G

In [44]:
print(len(new_geo_column))
print(len(new_PMID_column))
print(len(new_Title_column))
print(len(new_Authors_column))
print(len(new_human_col))
print(len(new_keywords_Col))
print(len(new_Score_col))
print(len(new_status_col))
print(len(new_link_col))

136
136
136
136
136
136
136
136
136


In [47]:
#This is the table of GEO accessions that have a publication
COLUMN = ['GEO-Accession','PMID','Title','Authors','Human','Mouse','Keywords', 'Score', 'Status','Link to Portal']
Dic = {'GEO-Accession':new_geo_column,'PMID':new_PMID_column,'Title':new_Title_column,'Authors':new_Authors_column,'Human':new_human_col,'Mouse':new_mouse_col,'Keywords':new_keywords_Col,'Score':new_Score_col,'Status':new_status_col,'Link to Portal':new_link_col}


In [48]:
filter_df = pandas.DataFrame(Dic, columns=COLUMN)

In [49]:
writer = pandas.ExcelWriter('results6.xlsx')
filter_df.to_excel(writer,'sheet1')
new_df_t2.to_excel(writer,'sheet2')
writer.save()

In [ ]:
print(len(link_col))

In [ ]:
print(status_col)

In [ ]:
print(link_col)

In [ ]:
print(Title_col)

In [ ]:
print(res_title_list)

In [ ]:
# for i in Authors_col:
#     str_Authors_col = ','.join(str(e) for e in Authors_col)
# for i in Keyword_col: 
#     str_Keyword_col = ','.join(str(e) for e in Keyword_col)
    

In [ ]:
#Get the GEO information from GEO accession
gse = GEOparse.get_GEO('GSE71831')

In [ ]:
#Get metadata
metadata = gse.metadata
metadata.keys()

In [ ]:
#Get Data of interest
pubID_i = gse.metadata['pubmed_id']
contributors = gse.metadata['contributor']
summary = gse.metadata['summary']
exp_design = gse.metadata['overall_design']
exp_type = gse.metadata['type']

In [ ]:
#Getting infor from pubmed using the pubmed ID
pubID = str(pubID_i[0])
#Getting xml format
pubmed_URL_xml = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubID +"&retmode=xml"
print(pubmed_URL_xml)
response = requests.get(pubmed_URL_xml)
with open('file.xml', 'wb') as f:
    f.write(response.content)
tree = ET.parse('file.xml')


In [ ]:
#The Data is stored in the child of the root
# child = tree.getchildren()
# print(child)
root = tree.getroot()
for i in root:
    child = i
print(child)


In [ ]:
#The data is stored in the child of the root
#The info of interest is in the medlineCitation level
medlineCitation = 'MedlineCitation'
#This are the elements of interest in this datalevel:
Article = 'Article'
KeywordList = 'KeywordList' #each element only has a single item. no index needed with for loop


In [ ]:
#Elements of Interest in Article level:
AuthorList = 'AuthorList'  # there are several, each item has last name index = 0, first name = 1, and abbrev = 2
ArticleTitle = 'ArticleTitle' # there is only one object so index = 0
ArticleDate = 'ArticleDate' # there are several, each item has a Year index = 0, Month index = 0, and day index = 3

In [ ]:
for i in child:
    if medlineCitation in str(i):
        for item in i:
            print(item)
            if Article in str(item):
                for ele in item:
                    print(ele)
                    if ArticleTitle in str(ele):
                        Article_Title = ele.text
                        print(Article_Title)
                        
                    if ArticleDate in str(ele):
                        Article_Year = ele[0].text
                        print(Article_Year)
                            
                    if AuthorList in str(ele):
                        Author_List = []
                        for author in ele:
                            Lname = author[0].text
                            Iname = author[1].text
                            complName = Lname + ', ' + Iname
                            Author_List.append(complName)
                        print(Author_List)
                        
            
            if KeywordList in str(item):
                keyword_list = []
                for ele in item:
                    keyword = ele.text
                    keyword_list.append(keyword)
                print(keyword_list)
    #Getting Info from the GEO website
    URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + 'GSE71831'
    print(URL_GEO_web)
    
    response2 = requests.get(URL_GEO_web)
    my_file = response2.content.decode()
    #Organism Key Word:
    mouse1 = 'Mus musculus'
    human1 = 'Homo sapiens'

    if mouse1 in my_file:
        is_mouse = 'yes'
        
    if human1 in my_file:
        print('human')
        is_human = 'yes'
    


In [ ]:
Author_List

In [ ]:
Article_Year

In [ ]:
Article_Title


In [ ]:
keyword_list

In [ ]:
#Getting Info from the GEO website
URL_GEO_web = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=" + GEOacc
print(URL_GEO_web)

In [ ]:
response2 = requests.get(URL_GEO_web)
my_file = response2.content.decode()


In [ ]:
#Organism Key Word:
mouse1 = 'Mus musculus'
human1 = 'Homo sapiens'

if mouse1 in my_file:
    print('mouse')
    is_mouse = 'yes'
if human1 in my_file:
    print('human')
    is_human = 'yes'

In [ ]:
#New Table
pubID_col = []
Title_col = []
Authors_col = []
Technique_col = []
Human_col = []
Mouse_col = []
Keyword_col = []
year_col = []
score_col = []

new_df = pandas.DataFrame({'pubID':pubID_col, 'title':Title_col}, columns=['pubID', 'title'])

In [50]:
print(year_col)

['2018', '2017', '2018', '2018', '2016', '2016', '2018', '2017', '2017', '2016', '2015', '2015', '2015', '2015', '2013', '2013', '2012', '2012', '2012', '2016', '2018', '2018', '2018', '2018', '2018', '2017', '2018', '2018', '2018', '2018', '2018', '2017', '2017', '2018', '2017', '2017', '2017', '2017', '2016', '2015', '2016', '2016', '2016', '2016', '2016', '2015', '2013', '2015', '2013', '2013', '2012', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2017', '2017', '2017', '2017', '2015', '2015', '2015', '2013', '2013', '2014', '2012', '2018', '2018', '2018', '2017', '2017', '2017', '2017', '2017', '2015', '2015', '2015', '2018', '2018', '2014', '2014', '2015', '2016', '2014', '2017', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018', '2018',

In [51]:
print(len(year_col))

162
